In [1]:
%load_ext bigdata
%hive_start
%timeout 300

In [2]:
!hdfs dfs -mkdir /tmp/drivers

mkdir: `/tmp/drivers': File exists


In [3]:
!hdfs dfs -ls /tmp/drivers/*

ls: `/tmp/drivers/*': No such file or directory


In [4]:
!hdfs dfs -tail /tmp/drivers/tbl0.csv
!hdfs dfs -tail /tmp/drivers/tbl1.csv

tail: `/tmp/drivers/tbl0.csv': No such file or directory
tail: `/tmp/drivers/tbl1.csv': No such file or directory


In [5]:
%%hive
DROP TABLE IF EXISTS tbl0;
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
--
DROP TABLE IF EXISTS tbl1;
CREATE TABLE tbl1 (
    c1 INT,
    c2 INT,
    c3 STRING,
    c4 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl1.csv' INTO TABLE tbl1; 

DROP TABLE IF EXISTS tbl0;
OK
Time taken: 12.521 seconds
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 3.005 seconds
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
Loading data to table default.tbl0
OK
Time taken: 4.185 seconds
--
DROP TABLE IF EXISTS tbl1;
OK
Time taken: 0.014 seconds
CREATE TABLE tbl1 (
    c1 INT,
    c2 INT,
    c3 STRING,
    c4 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 0.14 seconds
LOAD DATA LOCAL INPATH 'tbl1.csv' INTO TABLE tbl1; 
Loading data to table default.tbl1
OK
Time taken: 1.344 seconds


In [7]:
%%hive
SELECT * FROM tbl0;
SELECT * FROM tbl1;

SELECT * FROM tbl0;
OK
1	D	4	2016-06-25	["a","e","c","d"]	{"bb":10,"dd":20,"cc":40}
2	C	4	2015-05-14	["a","c"]	{"dd":40,"bb":20,"cc":10}
3	D	6	2014-12-26	["b","d"]	{"aa":10,"bb":40}
4	D	5	2016-06-25	["a","c","e","b","d"]	{"bb":40,"dd":20,"aa":10,"cc":30}
5	C	7	2016-05-23	["d","e","c"]	{"cc":10,"aa":20}
6	A	2	2018-06-14	["a","d"]	{"aa":20}
7	B	3	2014-12-22	["a","e","d"]	{"cc":20}
8	C	6	2015-08-20	["d","a","c","e"]	{"cc":10,"aa":20}
9	B	3	2017-12-08	["b","a","c","e"]	{"cc":40,"dd":10,"aa":30,"bb":20}
10	B	7	2015-07-28	["c","d","e","a","b"]	{"aa":10,"dd":40,"cc":30}
Time taken: 0.153 seconds, Fetched: 10 row(s)
SELECT * FROM tbl1;
OK
1	7	bb	{"A":0,"B":4,"C":1,"D":3,"E":5}
2	13	Cc	{"A":4,"B":1,"C":0,"D":5,"E":2}
3	11	dD	{"A":5,"B":4,"C":3,"D":1,"E":0}
4	3	BB	{"A":2,"B":4,"C":3,"D":5,"E":1}
5	14	cc	{"A":4,"B":0,"C":2,"D":5,"E":3}
6	8	cC	{"A":4,"B":5,"C":1,"D":2,"E":3}
7	2	DD	{"A":0,"B":5,"C":2,"D":4,"E":3}
8	6	dd	{"A":4,"B":2,"C":5,"D":3,"E":0}
9	9	aa	{"A":1,"B":4,"C":2,"D":5,"E":3}
10	4	Bb

In [8]:
%%hive
SELECT
    t1.c1,
    t1.c2,
    t2.c4
FROM
    tbl0
    t1
JOIN (
    SELECT
        c1,
        c4
    FROM
        tbl1
    ) t2
ON
    (t1.c1 = t2.c1);
    
INSERT OVERWRITE LOCAL DIRECTORY 'output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT tbl0.c1, tbl0.c2, tbl1.c4[tbl0.c2]
FROM tbl0, tbl1
WHERE (tbl0.c1=tbl1.c1);

SELECT
    t1.c1,
    t1.c2,
    t2.c4
FROM
    tbl0
    t1
JOIN (
    SELECT
        c1,
        c4
    FROM
        tbl1
    ) t2
ON
    (t1.c1 = t2.c1  );
Query ID = root_20200208224833_c752fbdb-e20d-4d57-b7bb-d4ffa0c769c6
Total jobs = 1
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
2020-02-08 22:48:46	Starting to launch local task to process map join;	maximum memory = 477626368
2020-02-08 22:48:49	Dump the side-table for tag: 1 with group count: 10 into file: file:/tmp/root/4518d040-67b6-4730-82dc-1e83423e3bd6/hive_2020-02-08_22-48-33_521_6526420827255028709

In [56]:
%%hive
--SELECT
  --  tbl0.c1,
    --tbl0.c2,
    --tbl1.c4['Key']
--FROM
  --  tbl0,
    --tbl1
--WHERE
    --tbl1.c4['key']='D';
    --(tbl0.c1 = tbl1.c1) and (tbl1.c4['key']='D');
  --  tbl0.c1 = tbl1.c1 and tbl0.c2=tbl1.c4['key'];
    
--SELECT tbl0.c1, tbl0.c2, tbl1.c4[tbl0.c2]
--FROM tbl0, tbl1
--WHERE (tbl0.c1=tbl1.c1) and (tbl1.c4['A']=0);
--WHERE (tbl0.c1=tbl1.c1);

INSERT OVERWRITE LOCAL DIRECTORY 'output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT tbl0.c1, tbl0.c2, tbl1.c4[tbl0.c2]
FROM tbl0, tbl1
WHERE (tbl0.c1=tbl1.c1);

--SELECT
  --  tbl0.c1,
    --tbl0.c2,
    --tbl1.c4['Key']
--FROM
  --  tbl0,
    --tbl1
--WHERE
    --tbl1.c4['key']='D';
    --(tbl0.c1 = tbl1.c1) and (tbl1.c4['key']='D');
  --  tbl0.c1 = tbl1.c1 and tbl0.c2=tbl1.c4['key'];
--SELECT tbl0.c1, tbl0.c2, tbl1.c4[tbl0.c2]
--FROM tbl0, tbl1
--WHERE (tbl0.c1=tbl1.c1) and (tbl1.c4['A']=0);
--WHERE (tbl0.c1=tbl1.c1);
INSERT OVERWRITE LOCAL DIRECTORY 'output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT tbl0.c1, tbl0.c2, tbl1.c4[tbl0.c2]
FROM tbl0, tbl1
WHERE (tbl0.c1=tbl1.c1);
Query ID = root_20200209001642_13ddd8bc-060f-4fcf-b8c4-2987f5d03894
Total jobs = 1
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings fo

In [160]:
%%hive
SELECT  fec, word ,count(*) FROM (
SELECT substr (c4,1,4) AS fec , word
FROM tbl0 lateral view explode(split(c4, '\\:')) temp_table AS word
) x
GROUP BY  fec ,word;

ON
    (t1.c1 = t2.c1  );
Query ID = root_20200208191822_950b3a85-d81c-4afd-b7ea-d3c5d11f7ef1
Total jobs = 1
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
2020-02-08 19:18:32	Starting to launch local task to process map join;	maximum memory = 477626368
2020-02-08 19:18:35	Dump the side-table for tag: 1 with group count: 10 into file: file:/tmp/root/51b149e5-d692-4c3a-8ded-97305de30c09/hive_2020-02-08_19-18-22_149_4478622126870134815-1/-local-10004/HashTable-Stage-3/MapJoin-mapfile291--.hashtable
2020-02-08 19:18:35	Uploaded 1 File to: file:/tmp/root/51b149e5-d69